In [2]:
import numpy as np
import pandas as pd
import datetime

In [3]:
"""pandas硬背。
dataframe.to_csv存储时默认的encoding是utf8，如果有汉字且在excel打开时请指定GB2312。
dataframe存储时默认加一列index，一般需指定index=False。
dataframe存储时默认header=True，当header=False第一行变为header。注意header=n有歧义，当n=0时等价于header=False，当n!=0时等价于header=True，注意这个n不是指的第几行。
pd.read_csv读取数据默认encoding是utf8。
pd.read_csv读取数据时默认在原始数据前加一列索引，0开始递增。
pd.read_csv读取数据时默认header=0，也就是原始数据第一行（通常是列名）。如果指写header=3则以第4行作为列名，且跳过前三行数据不读取（包括列名）。注意header不能赋值True或False。
"""

"""时间解析函数"""
pd.to_datetime(['1/22/2018', np.datetime64('2018-01-22'),datetime.datetime(2018, 1, 22)])

DatetimeIndex(['2018-01-22', '2018-01-22', '2018-01-22'], dtype='datetime64[ns]', freq=None)

# dataframe

### dataframe创建

In [4]:
df = pd.DataFrame(np.arange(6).reshape(2, 3), columns=list('abc'), index=range(11,13), dtype='int')  # the dtype parameter assignment all columns

In [5]:
sr1 = pd.Series(list('xyz'),index=range(3))
df = pd.DataFrame([[1,2,3],(11,12,13),sr1],columns=list('abc'),index=range(11,14), dtype='int')  # 元组同理

In [9]:
sr1 = pd.Series(list('abc'),index=range(3))
sr2 = pd.Series(list('bcd'),index=range(1,4))
df = pd.DataFrame({'xa':sr1,'xb':sr2,'xc':[11,22,33,44],'xd':(6,7,8,9)})  # 通过字典创建时，若指定columns，冲突时以columns为准，仅columns有的设置为空列，仅字典有的不加入dataframe

### dataframe columns操作

In [10]:
df1 = pd.DataFrame(np.arange(6).reshape(2, 3), columns=list('abc'), index=range(11,13), dtype='int')  # the dtype parameter assignment all columns

In [11]:
df1.columns.name  # defult None

In [12]:
df1.columns.name = 'wanghl'  # assignment
df1.columns.name

'wanghl'

In [13]:
df1.columns  # return index

Index(['a', 'b', 'c'], dtype='object', name='wanghl')

In [14]:
df1.columns = list('xyz'); df1  # modify in place

,x,y,z
11,0,1,2
12,3,4,5


In [15]:
list(df1)  # return list

['x', 'y', 'z']

In [16]:
df2 = df1.add_prefix('w_'); df2  # create new

,w_x,w_y,w_z
11,0,1,2
12,3,4,5


### dataframe index操作

In [17]:
df1 = pd.DataFrame(np.arange(6).reshape(2, 3), columns=list('abc'), index=range(11,13), dtype='int')  # the dtype parameter assignment all columns

In [18]:
df1.index.name  # default None

In [19]:
df1.index.name = 'wanghl'
df1.index.name

'wanghl'

In [20]:
df1.index  # return index

RangeIndex(start=11, stop=13, step=1, name='wanghl')

In [21]:
df1.index = list('ab'); df1  # modify in place

,a,b,c
a,0,1,2
b,3,4,5


### copy操作

In [22]:
df1 = pd.DataFrame(np.arange(6).reshape(2, 3), columns=list('abc'), index=range(11,13), dtype='int')  # the dtype parameter assignment all columns

In [23]:
df2 = df1; df2['a'] = 66; df1  # df1 and df2 point to the same location

,a,b,c
11,66,1,2
12,66,4,5


In [24]:
df2 = df1.copy()  # df2 and df1 point to different location

### slice操作

In [25]:
df1 = pd.DataFrame([[4,5,6],[44,55,66]],columns=[1, 2, 'c'], index=range(1,3), dtype='int')  # the dtype parameter assignment all columns

In [26]:
"""df1.loc[:, 'c']"""
df1.c  # by columns, label based(only string), return series
"""df1.loc[:, 1]"""
df1[1]  # by columns, label based, return series
"""df1.loc[:, [1]]"""
df1[[1]]  # by columns, label based, return dataframe
"""df1.iloc[:1]"""
df1[:1]  # by rows, positional based, return dataframe
"""df1.iloc[[1]]"""
df1.take([1])  # by rows, positional based(can't use ":"), return dataframe
"""df1.loc[1]"""
df1.xs(1)  # by row(one row each time), label based, return series

1    4
2    5
c    6
Name: 1, dtype: int32

In [27]:
"""filter支持正则表达示切片。修改切片后数据对原dataframe无影响。"""
df1 = pd.DataFrame(np.arange(9).reshape(3, 3), columns=['ca','cb','x'], index=['ia','ib','y'], dtype='int')

In [28]:
df1.filter(items=['a','x'], axis=1)

,x
ia,2
ib,5
y,8


In [29]:
df1.filter(like='c', axis=1)  # 包含c的列

,ca,cb
ia,0,1
ib,3,4
y,6,7


In [30]:
df1.filter(regex='i+', axis=0)  # 正则匹配

,ca,cb,x
ia,0,1,2
ib,3,4,5


In [31]:
"""利用difference来排除不需要的行或列。修改切片后数据对原dataframe无影响。这里index.difference未举例，同理"""
df1 = pd.DataFrame(np.arange(9).reshape(3, 3), columns=['ca','cb','x'], index=['ia','ib','y'], dtype='int')

In [32]:
df1.columns.difference(['ca','x'])

Index(['cb'], dtype='object')

In [33]:
df1[df1.columns.difference(['ca','x'])]

,cb
ia,1
ib,4
y,7


### loc和iloc操作

In [34]:
dfloc = pd.DataFrame([[1,2,3],[11,22,33],[111,222,333]], columns=list('zxy'), index=list('cab'))  # the dtype parameter assignment all columns

In [35]:
dfloc.loc['a']

z    11
x    22
y    33
Name: a, dtype: int64

In [36]:
dfloc.loc[['a']]

,z,x,y
a,11,22,33


In [37]:
dfloc.loc[:'a']

,z,x,y
c,1,2,3
a,11,22,33


In [38]:
dfloc.loc[:,'x']

c      2
a     22
b    222
Name: x, dtype: int64

In [39]:
dfloc.loc[:,['x']]

,x
c,2
a,22
b,222


In [40]:
dfloc.loc[:,:'x']

,z,x
c,1,2
a,11,22
b,111,222


In [41]:
dfloc.loc['a','x']

22

In [42]:
dfloc.loc[['a','a'],['x','x']]

,x,x
a,22,22
a,22,22


In [43]:
dfloc.loc[:'a',:'x']

,z,x
c,1,2
a,11,22


In [44]:
dfloc.loc['a',['x','x']]

x    22
x    22
Name: a, dtype: int64

In [45]:
dfloc.loc['a',:'x']

z    11
x    22
Name: a, dtype: int64

In [46]:
dfloc.loc[:'a',['x','x']]

,x,x
c,2,2
a,22,22


In [47]:
"""关于使用loc索引数值时的两个坑"""
"""即使str(9)作为索引，dataframe也会按照number来处理，与9无区别"""
dfloc_ = pd.DataFrame([[1,2,3],[11,22,33],[111,222,333]], columns=[6, 'a', str(9)], index=[11, 11, 22])  # mixed index of number and string 

In [48]:
"""坑1：当行列索引为纯number组成时，并且按位置排序时索引的number不减少（递增或不减）时，则帽号前后的数字可不在现有索引中，dataframe会按数值来比较大小进行切片。
如果纯number组成的索引出现递减的情况，帽号前后使用索引之外的数字会报错KeyError"""
dfloc_.loc[1:12]

,6,a,9
11,1,2,3
11,11,22,33


In [49]:
"""坑2：当两种条件同时满足会报错：数字和字母混合索引，帽号前后是数字。比如df2.loc[:,:9], 会报错TypeError"""
dfloc_.loc[:11,:'a']

,6,a
11,1,2
11,11,22


In [50]:
dfiloc = pd.DataFrame([[4,5,6],[44,55,66]], columns=list('xyz'), index=list('ab')); dfiloc

,x,y,z
a,4,5,6
b,44,55,66


In [51]:
dfiloc.iloc[1]

x    44
y    55
z    66
Name: b, dtype: int64

In [52]:
dfiloc.iloc[[1]]

,x,y,z
b,44,55,66


In [53]:
dfiloc.iloc[:1]

,x,y,z
a,4,5,6


In [54]:
dfiloc.iloc[:,1]

a     5
b    55
Name: y, dtype: int64

In [55]:
dfiloc.iloc[:,[1]]

,y
a,5
b,55


In [56]:
dfiloc.iloc[:,:1]

,x
a,4
b,44


In [57]:
dfiloc.iloc[1,1]

55

In [58]:
dfiloc.iloc[[1,1],[1,1]]

,y,y
b,55,55
b,55,55


In [59]:
dfiloc.iloc[:1,:1]

,x
a,4


In [60]:
dfiloc.iloc[1, [1, 1]]

y    55
y    55
Name: b, dtype: int64

In [61]:
dfiloc.iloc[1, :1]

x    44
Name: b, dtype: int64

In [62]:
dfiloc.iloc[:1, [1,1]]

,y,y
a,5,5


### at和iat操作

In [63]:
df1 = pd.DataFrame([[4,5,6],[44,55,66]], columns=[1, 2, 'c'], index=range(1,3), dtype='float')  # the dtype parameter assignment all columns

In [64]:
df1.at[1,1]  #  by cell, label based, return value

4.0

In [65]:
df1.iat[1,1]  #  by cell, positional based, return value

55.0

In [66]:
df1.at[1,1] = 111; df1  # modify in place, label based

,1,2,c
1,111.0,5.0,6.0
2,44.0,55.0,66.0


In [67]:
df1.iat[1,1] = 222; df1  # modify in place, positional based

,1,2,c
1,111.0,5.0,6.0
2,44.0,222.0,66.0


### 赋值或新增

In [68]:
df1 = pd.DataFrame([[4,5,6],[44,55,66]],columns=[1, 2, 'c'], index=range(1,3), dtype='int')  # the dtype parameter assignment all columns

In [69]:
"""以下赋值均为in place, broadcast"""
df1.c = 'x'
df1[1] = 'x'
df1[[1]] = 'x'
df1[:1] = 'x'
df1.loc[1] = 'x'
df1.loc[[1]] = 'x'
df1.iloc[1] = 'x'
df1.iloc[[1]] = 'x'
"""注意at只改变1个值，这个新值的数据类型不能与原来列的数据类型冲突"""
df1.at[1,1] = 111
df1.iat[1,1] = 222
"""不可以赋值，因为是function call"""
df1.take([1])  # 对新得到的dataframe赋值，不会改变原来的df1
df1.xs(1)  # 对新得到的series赋值，会改变原来的df1

1    111
2      x
c      x
Name: 1, dtype: object

In [70]:
df1 = pd.DataFrame([[4,5,6,7],[44,55,66,77]],columns=[1, 2, 'c', 'd'], index=range(1,3), dtype='int')  # the dtype parameter assignment all columns

In [71]:
"""ndarray,list,tuple均不能广播，但是tuple一个元素的时候可以广播"""
df1.loc[1] = ['list','list','list','list']; df1
df1.loc[2] = ('tuple','tuple','tuple','tuple'); df1
df1.loc[0] = np.array(['np', 'np', 'np', 'np']); df1
df1.loc[:,'e'] = np.array(['np', 'np', 'np']); df1

,1,2,c,d,e
1,list,list,list,list,np
2,tuple,tuple,tuple,tuple,np
0,np,np,np,np,np


In [72]:
"""ndarray,list,tuple也可以按区块赋值"""
df1.loc[[1,2],['d','e']] = np.array([['nmpy','nmpy'],['nmpy','nmpy']])
df1.loc[[1,2],[1,2]] = [['lst','lst'],['lst','lst']]; df1

,1,2,c,d,e
1,lst,lst,list,nmpy,nmpy
2,lst,lst,tuple,nmpy,nmpy
0,np,np,np,np,np


In [73]:
"""dict, series按索引精确匹配，多出来的丢弃，缺失的填NaN"""
df1.loc[1] = {1:'dict',2:'dict',3:'dict'}
df1.loc[2] = pd.Series(['series','series','series'], index=[1,'c','e'])
df1

,1,2,c,d,e
1,dict,dict,NaN,NaN,NaN
2,series,NaN,series,NaN,series
0,np,np,np,np,np


In [74]:
"""Pandas赋值是精确匹配行列索引的，只测试了下面的例子"""
pd2 = pd.DataFrame({1:['pandas', 'pandas'], 'c':['pandas', 'pandas']}, index=[2,3])
df1.loc[:,[1,2,'c']] = pd2
df1

,1,2,c,d,e
1,NaN,NaN,NaN,NaN,NaN
2,pandas,NaN,pandas,NaN,series
0,NaN,NaN,NaN,np,np


In [76]:
"""新增行或列，只有df['a']和df.loc['a'],df.loc[:,'a']这两种方法"""
df1 = pd.DataFrame([[4,5,6],[44,55,66]],columns=[1, 2, 3], index=[1, 2], dtype='int')  # the dtype parameter assignment all columns

In [77]:
"""list,tuple,ndarray要长度完全匹配"""
df1['a']  = [1,1]
df1.loc[:,'b'] = np.array([1,1])
df1.loc[66] = (1,2,3,4,5)
df1

,1,2,3,a,b
1,4,5,6,1,1
2,44,55,66,1,1
66,1,2,3,4,5


In [78]:
"""series可以按索引匹配，dict无法用于新增行或列"""
df1.loc[:,'ab'] = pd.Series({1:'sr',8:'sr'})
df1.loc['xb'] = pd.Series({1:'sr',8:'sr'})
df1

,1,2,3,a,b,ab
1,4,5.0,6.0,1.0,1.0,sr
2,44,55.0,66.0,1.0,1.0,NaN
66,1,2.0,3.0,4.0,5.0,NaN
xb,sr,NaN,NaN,NaN,NaN,NaN


### 删除操作

In [79]:
df1 = pd.DataFrame([[4,5,6],[44,55,66]],columns=[1, 2, 'c'], index=range(1,3), dtype='int')  # the dtype parameter assignment all columns

In [80]:
df1.drop([1,'c'], axis=1)  # delete by label, create new, can multi column each time

,2
1,5
2,55


In [81]:
df1.drop([1], axis=0)  # delete by label, create new, can multi column each time

,1,2,c
2,44,55,66


In [82]:
"""del方法只能删除列"""
del df1[1]; df1  # delete by label, in place, one column each time

,2,c
1,5,6
2,55,66


### filter操作

In [83]:
df1 = pd.DataFrame([[4,5,6],[44,55,66]],columns=[1, 2, 'c'], index=range(1,3), dtype='int')  # the dtype parameter assignment all columns

In [84]:
"""
df1[df1['column'].isin([1,2,3,4])]
df1[df1['column'] >= 66]
"""
df1[[True, False]][[False]]  #  by rows, bool based, return dataframe

,1,2,c


In [85]:
df1[df1>10]  # filter by cell

,1,2,c
1,NaN,NaN,NaN
2,44.0,55.0,66.0


### value操作

In [86]:
df1 = pd.DataFrame([[4,5,6],[44,55,66]],columns=[1, 2, 'c'], index=range(1,3), dtype='int')  # the dtype parameter assignment all columns
df1.values  # return numpy

array([[ 4,  5,  6],
       [44, 55, 66]])

In [87]:
df1.values.tolist()  # tolist is a method of numpy 

[[4, 5, 6], [44, 55, 66]]

In [88]:
"""transposition"""
df1.T.values.tolist() == df1.values.T.tolist()

True

### dtype操作

In [93]:
df1 = pd.DataFrame([[4,5,6],[44,55,66]],columns=[1, 2, 'c'], index=range(1,3), dtype='float')  # the dtype parameter assignment all columns

In [94]:
df1.dtypes  # return series

1    float64
2    float64
c    float64
dtype: object

In [95]:
sr1 = df1[1].astype(int); sr1  # create new series, don't modify original dataframe

1     4
2    44
Name: 1, dtype: int32

In [96]:
df2 = df1.astype(int); df1  # create new dataframe, don't modify original dataframe

,1,2,c
1,4.0,5.0,6.0
2,44.0,55.0,66.0


### dataframe iter操作

In [97]:
df1 = pd.DataFrame(np.random.randn(2,3), columns=list('abc'), dtype='float')  # 好像所有列只能指定一种类型

In [98]:
list(df1.iterrows())  # return iterator by rows, (index, Series)

[(0, a    1.108081
  b   -0.759317
  c    0.647127
  Name: 0, dtype: float64), (1, a    1.259736
  b    0.637542
  c    1.011445
  Name: 1, dtype: float64)]

In [99]:
list(df1.itertuples())  # return iterator by rows, tuple

[Pandas(Index=0, a=1.1080810483230685, b=-0.7593167551329959, c=0.6471271355943526),
 Pandas(Index=1, a=1.2597358001839565, b=0.6375417935678254, c=1.0114448037669133)]

In [100]:
list(df1.iteritems())  # return iterator by columns, (column, Series)

[('a', 0    1.108081
  1    1.259736
  Name: a, dtype: float64), ('b', 0   -0.759317
  1    0.637542
  Name: b, dtype: float64), ('c', 0    0.647127
  1    1.011445
  Name: c, dtype: float64)]